<a href="https://colab.research.google.com/github/marwansalem/Registration-Login-Forms/blob/master/Copy_of_topology_mapping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CLVhkMfwJxeUit4qhkUsQWMu8rYHaAMr' -O dataset.zip
! unzip dataset.zip

In [0]:
! mkdir tens fifty hundred
! cp Dataset/*_10_* tens
! cp Dataset/*_50_* fifty
! cp Dataset/*_100_* hundred

In [0]:
! ls tens/

t_10_0.txt  t_10_2.txt	t_10_4.txt  t_10_6.txt	t_10_8.txt
t_10_1.txt  t_10_3.txt	t_10_5.txt  t_10_7.txt	t_10_9.txt


In [0]:
import numpy as np
import os
from pathlib import Path
import math
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
import networkx as nx
import matplotlib.pyplot as plt

In [0]:
def check_symmetric(a, tol=1e-8):
    return np.all(np.abs(a-a.T) < tol)

In [0]:
def normalize_rows(Mat):
    return Mat / np.linalg.norm(Mat, ord=2, axis=1, keepdims=True)

In [0]:
def get_similarity_matrix(folder, index, num_of_nodes):
    pathlist = Path(folder).glob('*.txt')
    files = sorted([str(path) for path in pathlist])
    # print(files)
    A = np.zeros(shape=(num_of_nodes, num_of_nodes))
    noise = 1e-5
    A += noise 
    f = open(files[index])
    lines = f.readlines()
    lines = [l.strip() for l in lines]
    for l in lines:
        assert(len(l.split()) == 3)
        u, v, w = [int(num) for num in l.split()]
        u -= 1
        v -= 1
        A[u][v] += w
        A[v][u] += w
    
    return A

In [0]:
def get_ground_truth():
    lines = []
    with open('Dataset//ground_truth.txt', 'r') as f:
        lines = f.readlines()

    partitions = [ [int(val) for val in line[:-1].split(' ')]  for line in lines]

    return (partitions[:10], partitions[10:])

labels10, labels50 = get_ground_truth()

In [0]:
def get_degree_matrix(A):
    n = len(A)
    Delta = np.zeros(shape=(n, n))
    for i in range(n):
        Delta[i][i] = np.sum(A[i])
    return Delta

In [0]:
def get_B(SimMat, DegMat):
    L = DegMat - SimMat
    Deg_inv = np.linalg.pinv(DegMat)
    return Deg_inv.dot(L)

In [0]:
def build_contingency_table(labels, true_labels):
    n_clusters = np.max(labels) + 1
    n_partitions = np.max(true_labels) + 1
    dims = (n_clusters , n_partitions)
    cont_tab = np.zeros((n_clusters, n_partitions))
    for i in range(len(labels)):
        ci = labels[i]
        pi = true_labels[i]
        cont_tab[ci][pi] += 1 #this point is in common between the cluster and the partition
    return cont_tab

In [0]:
def spectral_clustering(SimMat, DegMat, k):
    B = get_B(SimMat, DegMat)
    eigvals, eigvecs = np.linalg.eigh(B)
    U = eigvecs.T[:k].T
    Y = normalize_rows(U)
    kmeans = KMeans(n_clusters=k, random_state=0).fit(Y)
    return kmeans.labels_

In [0]:
def cond_entropy(contingency_tab, labels, ground_truth):
    r = np.max(labels) + 1 # number of clusters
    k = np.max(ground_truth) + 1 #number of partitions
    cluster_sizes = [0] * r # sizes of clusters Ci (ni)
    partition_sizes = [0] * k  # sizes of partitions Tj(mj)  
    num_of_points = len(labels)

    for ci in labels:
        cluster_sizes[ci] += 1
    for tj in ground_truth:
        partition_sizes[tj] += 1

    h_t_c = 0
    #H(T/c) = H(C,T) - H(C)
    # compute first H(C,T)
    for i in range(r):
        for j in range(k):
            #pij = nij /n , nij : cont_tab[i][j]
            p_i_j = contingency_tab[i][j] / num_of_points
            if p_i_j != 0: # to avoid log2(0)
                h_t_c -= p_i_j * np.log2(p_i_j)
                print(h_t_c)
    # compute H(C)
    for i in range(r):
        pci = cluster_sizes[i]/num_of_points
        if pci !=0:# to avoid division by zero
            h_t_c += pci*np.log2(pci)
            print(h_t_c)
    return h_t_c


In [0]:
def compute_traffic(mat, labels, noise=0):
    n, n = np.shape(Mat)
    internal_traffic = np.zeros(np.max(labels) +1)
    external_traffic = np.zeros(np.max(labels)+1)
    for i in range(n): # iterate on elements below main diagonal
        for j in range(i):
            w = mat[i][j]

            if w <= noise: #ignore noise
                continue

            if labels[i] == labels[j]: # if they have same label then they are in the same cluster
                cluster_num = labels[i]
                #add weight of edge to the internal traffic of the cluster
                internal_traffic[cluster_num] += w
            else:
                #not in same cluster, so add the weight of the edge to external traffic 
                external_traffic[labels[i]] += w
                external_traffic[labels[j]] += w
    
    return internal_traffic, external_traffic

In [0]:
def get_distance_matrix(mat, noise=0):
    dims = np.shape(mat)
    dist_mat = np.ones(dims)
    n = len(mat)
    for i in range(n):
        for j in range(i):
            w = mat[i][j]
            if w>= noise:
                dist_mat[i][j] = dist_mat[j][i] = np.exp(-0.01* w)
    
    return dist_mat

In [0]:
def normalized_cut_measure(weight_mat,labels , noise=0):
    #1. convert weight matrix to distance matrix e^-0.01w
    dist = get_distance_matrix(weight_mat,noise= noise)
    #2 get internal distances within cluster, and among clusters
    internal_dist, external_dist = compute_traffic(dist, labels, noise=noise)

    NC = 0
    for i in range(len(internal_dist)):
        NC +=  external_dist[i]/(external_dist[i] + internal_dist[i])
    
    return NC

In [0]:
def f_measure_onerow(cont_mat, row : int):
    arr = cont_mat[row]
    max_val = np.amax(arr)
    index = np.where(arr == max_val)[0][0]
    return (2 * cont_mat[row][index]) / (np.sum(cont_mat[row]) + np.sum(cont_mat[ : , index]))

In [0]:
def f_measure(cont_table):
    r = cont_table.shape[0]
    return (1.0 / r) * (np.sum([f_measure_onerow(cont_table, i) for i in range(r)]))

In [0]:
# for visualizing graph
def draw_graph(A, clusters=None, title='' ,noise=0):
    G =nx.Graph()
    n, n = np.shape(A)
    for i in range(n):
        for j in range(i): #works as long as entrys a[i][i] are empty!
            #print(str(i) +' ,' + str(j) )
            if A[i][j] > noise:
                G.add_edge(i+1,j+1, weight = A[i][j])
            else:# add node if it doesnt exist
                G.add_node(i+1)
                
    pos = nx.spring_layout(G)
    #pos = nx.
    plt.figure()
    plt.axis('off')
    if clusters : 
        colors =[clusters[i-1] for i in G.nodes  ]
        nx.draw(G,pos=pos, node_color=colors, with_labels = True)
    else:
        nx.draw(G,pos=pos, with_labels= True)
    labels = nx.get_edge_attributes(G,'weight')# get weights
    nx.draw_networkx_edge_labels(G,pos, edge_labels=labels)# draw weights on edges
    plt.title(title)
    plt.show()

In [0]:
folder_names = ['tens', 'fifty', 'hundred']
sizes = [10, 50, 100]
kvals_10 = [2, 4, 6, 8, 10]
kvals_50_100 = [2,10,12,15,17,20,25]

In [0]:
# print(DegreeMatrix)
# print(B)
# print(SimilarityMatrix)

In [0]:
now = 0 # 0 -> 10, 1 ->50 2 -> 100
folder_name = folder_names[now]
num_of_nodes = sizes[now]
noise = 1e-5
n_clusters = kvals_10[3]  # choose a number, ba3'ayar fiha wana ba-test

def run():
    for i in range(10):
        for n_clusters in kvals_10:
            file_idx = i
            SimilarityMatrix = get_similarity_matrix(folder=folder_name, index=file_idx, num_of_nodes=num_of_nodes)
            DegreeMatrix = get_degree_matrix(SimilarityMatrix)
            k_means_labels = spectral_clustering(SimilarityMatrix, DegreeMatrix, n_clusters)
            title = 'file:{},K={}'.format(file_idx, n_clusters)
            # draw_graph(SimilarityMatrix, clusters= list(k_means_labels), title=title, noise = noise)
            true_labels = None
            n = len(DegreeMatrix)
            if n == 10:
                true_labels = labels10
            elif n == 50:
                true_labels = labels50
            elif n == 100:
                print('true label = None')
            
            cont_table = build_contingency_table(k_means_labels, true_labels)
            print(f'f-measure for cluster {n_clusters}: ', f_measure(cont_table))
            # clustering = SpectralClustering(n_clusters=n_clusters,
            #     assign_labels="discretize",
            #     random_state=0).fit(SimilarityMatrix)
            # print('sc : ', clustering.labels_, '\n\n')
        print('\n')

run()

f-measure for cluster 2:  0.4380952380952381
f-measure for cluster 4:  0.3074786324786325
f-measure for cluster 6:  0.22895299145299147
f-measure for cluster 8:  0.18842095404595405
f-measure for cluster 10:  0.16076839826839828


f-measure for cluster 2:  0.4258064516129032
f-measure for cluster 4:  0.30627705627705626
f-measure for cluster 6:  0.23362332112332113
f-measure for cluster 8:  0.18990904928404928
f-measure for cluster 10:  0.16076839826839828


f-measure for cluster 2:  0.4307692307692308
f-measure for cluster 4:  0.30627705627705626
f-measure for cluster 6:  0.23511904761904762
f-measure for cluster 8:  0.18973214285714285
f-measure for cluster 10:  0.16076839826839828


f-measure for cluster 2:  0.39166666666666666
f-measure for cluster 4:  0.30629084967320264
f-measure for cluster 6:  0.22998575498575502
f-measure for cluster 8:  0.18483946608946608
f-measure for cluster 10:  0.16076839826839828


f-measure for cluster 2:  0.4076923076923077
f-measure for cluster 4:  0